In [101]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

# Otras
from tqdm import tqdm
import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep
import random
import asyncio
import aiohttp
import re

import sys
import os
src_path = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path.append(src_path)
from support import extraccionurlsasincrona

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [70]:
# driver=webdriver.Chrome()
# url_facua = "https://super.facua.org/"
#     # Abrimos navegador y maximizamos página en la url
# driver.get(url_facua)
# driver.maximize_window()
# sleep(2)
#     # Aceptacion de cookies
# try:
#     driver.find_element("css selector","#rcc-confirm-button").click()
# except:    
#     print('No encuentro el boton')
# sleep(2)
# html_table_page = driver.page_source
# sopa = BeautifulSoup(html_table_page, 'html.parser')
# supermercados=['mercadona','carrefour','eroski','dia','hipercor','alcampo']
# urls = [link["href"] for link in sopa.find_all("a", href=True) if any(keyword in link["href"] for keyword in supermercados)]
# print(urls)

Creacion de Urls básicas de extraccion

In [19]:
url_facua = "https://super.facua.org/"

# Urls supermercados
supermercados=['mercadona','carrefour','eroski','dia','hipercor','alcampo']
urls_supermercados = [f"{url_facua}{keyword}/" for keyword in supermercados]

# Urls aceite girasol
producto='aceite-de-girasol'
urlsgirasol = [f"{url}{producto}/" for url in urls_supermercados]

# Urls aceite aceite de oliva
producto='aceite-de-oliva'
urlsoliva = [f"{url}{producto}/" for url in urls_supermercados]
tiposubproductoaceiteoliva=['suave-e-intenso','virgen','virgen-extra']
urlsolivasubtipos=[]
for producto2 in tiposubproductoaceiteoliva:
    urlsolivasubtipos.extend([f"{url}{producto2}/" for url in urlsoliva])

# Urls aceite aceite de oliva
producto='leche'
urlsleche = [f"{url}{producto}/" for url in urls_supermercados]
tiposubproductoleche=['enriquecida','entera-semi-desnatada','sin-lactosa']
urlslecheubtipos=[]
for producto2 in tiposubproductoleche:
    urlslecheubtipos.extend([f"{url}{producto2}/" for url in urlsleche])

# Juntar todas las urls en una lista
Listaurls=urlsgirasol+urlsolivasubtipos+urlslecheubtipos

In [20]:
Listaurls

['https://super.facua.org/mercadona/aceite-de-girasol/',
 'https://super.facua.org/carrefour/aceite-de-girasol/',
 'https://super.facua.org/eroski/aceite-de-girasol/',
 'https://super.facua.org/dia/aceite-de-girasol/',
 'https://super.facua.org/hipercor/aceite-de-girasol/',
 'https://super.facua.org/alcampo/aceite-de-girasol/',
 'https://super.facua.org/mercadona/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/carrefour/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/eroski/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/dia/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/hipercor/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/alcampo/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/mercadona/aceite-de-oliva/virgen/',
 'https://super.facua.org/carrefour/aceite-de-oliva/virgen/',
 'https://super.facua.org/eroski/aceite-de-oliva/virgen/',
 'https://super.facua.org/dia/aceite-de-oliva/virgen/',
 'https://super.f

Busqueda asincrona de Urls

In [48]:
# Función asíncrona para obtener el contenido de una URL y procesarlo con BeautifulSoup
async def search(url, session):
    async with session.get(url) as response:
        html = await response.text()
        return BeautifulSoup(html, "html.parser")  # Retorna la sopa de cada URL

# Función principal para manejar las peticiones asíncronas
async def main(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [search(url, session) for url in urls]
        results = await asyncio.gather(*tasks)
        return results
    # Ejecución de corutinas en entornos interactivos

listasopas = await main(Listaurls)
    


In [51]:
soopas=extraccionurlsasincrona(Listaurls)

In [54]:
print(soopas)

<coroutine object extraccionurlsasincrona at 0x000001B52ABB8D60>


In [72]:
# Encuentra todos los enlaces que coincidan con el patrón
listatotalurlsproductos=[]

for sopa in listasopas:
    # Encuentra todas las etiquetas <a> con la clase 'btn-unirme'
    enlaces = sopa.find_all('a', class_=lambda c: c and 'btn-unirme' in c)
    
    # Extrae el atributo 'href' de cada enlace encontrado y agrégalo a la lista si existe
    for enlace in enlaces:
        href = enlace.get('href')
        if href and href.startswith('https://'):
            if href.endswith('unirme/'):
                None
            else:
                listatotalurlsproductos.append(href)
listatotalurlsproductos

['https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-1-l/',
 'https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-capicua-garrafa-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-carrefour-classic-1-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-carrefour-garrafa-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-coosol-1-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-especial-para-freir-capicua-3-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-fontasol-1-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-fontasol-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-koipe-fritos-1-l/',
 'https://super.facua.org/carrefour/aceite-de-gi

In [88]:
len(listatotalurlsproductos)

1531

In [98]:
listatotalurlsproductos[0]

'https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-1-l/'

In [99]:
listacorta=listatotalurlsproductos[:30]
len(listacorta)

30

In [103]:
listasopasproductos=[]
for url in tqdm(listacorta,desc="Procesando URLs"):
    sopa=BeautifulSoup(url, "html.parser")
    listasopasproductos.append(sopa)

TypeError: 'module' object is not callable. Did you mean: 'tqdm.tqdm(...)'?

In [86]:
async def search(url, session):
    async with session.get(url) as response:
        html = await response.text()
        return BeautifulSoup(html, "html.parser")  # Retorna la sopa de cada URL

# Ejecuta las tareas sin un bucle explícito
async with aiohttp.ClientSession() as session:
    resultados = await asyncio.gather(*[search(url, session) for url in listacorta])

[
 <!DOCTYPE html>
 
 <html lang="es-ES">
 <head>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
 <meta content="super.FACUA.org" name="author"/>
 <meta content="FACUA vigila el precio de Aceite de girasol refinado 0,2º Hacendado 1 l. en Mercadona para ti" name="description"/>
 <meta content="super.FACUA.org" name="author"/>
 <meta content="precio Aceite de girasol refinado 0,2º Hacendado 1 l. en Mercadona" name="Keywords"/>
 <meta content="all" name="googlebot"/>
 <meta content="index" name="googlebot"/>
 <meta content="follow" name="googlebot"/>
 <meta content="all" name="robots"/>
 <meta content="index" name="robots"/>
 <meta content="follow" name="robots"/>
 <title>Precio HOY en Mercadona de Aceite de girasol refinado 0,2º Hacendado 1 l.</title>
 <meta content="index,follow" name="robots"/>
 <link href="https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-1-l/" rel="canonical"/

In [81]:
# Función asíncrona para obtener el contenido de una URL y procesarlo con BeautifulSoup
async def search(url, session):
    async with session.get(url) as response:
        html = await response.text()
        return BeautifulSoup(html, "html.parser")  # Retorna la sopa de cada URL

# Función principal para manejar las peticiones asíncronas
async def main(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [search(url, session) for url in urls]
        results = await asyncio.gather(*tasks)
        return results
    # Ejecución de corutinas en entornos interactivos

listasopastablas = await main(listatotalurlsproductos)
    
    

TimeoutError: 